In [ ]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from Bio import SeqIO
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
import random
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef, make_scorer
from statistics import stdev, variance, mean

In [ ]:
client_set=set()
others_set=set()
for rec in SeqIO.parse("../fig1/result/drllps_client_clstr_Homo_sapiens.fasta", "fasta"):
    client_set.add(rec.id)
for rec in SeqIO.parse("../fig1/result/drllps_nonllps_clstr_Homo_sapiens.fasta", "fasta"):
    others_set.add(rec.id)
    
mat=np.load("PTT5XLU50_human.npy", allow_pickle=True)
mat=mat.item()

list_client=[]
list_others=[]
for k in mat.keys():
    if k in others_set:
        list_others.append(mat[k])
    elif k in client_set:
        list_client.append(mat[k])

In [ ]:
class Display_auc():
    def __init__(self,x:np.array, y:np.array, model):
        self.x=x
        self.y=y
        self.model=model
        
    def cv(self, x, y):
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        out_y=[]
        out_y_proba=[]
        scores_rocauc=[]
        scores_prauc=[]
        scores_mcc=[]
        tprs = []
        mean_fpr = np.linspace(0, 1, 1000)
        
        for train,test in cv.split(x, y):
            self.model.fit(x[train],y[train])
            y_pred_proba=model.predict_proba(x[test])[:,1]
            y_pred=model.predict(x[test])
            fpr, tpr, thresholds = roc_curve(y[test], y_pred_proba)
            roc_auc=auc(fpr,tpr)
            precision, recall, thresholds = precision_recall_curve(y[test], y_pred_proba)
            pr_auc = auc(recall, precision)
            mcc=matthews_corrcoef(y[test], y_pred)
            
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            
            scores_rocauc.append(roc_auc)
            scores_prauc.append(pr_auc)
            scores_mcc.append(mcc)
        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        return mean_tpr, mean(scores_rocauc), mean(scores_prauc), mean(scores_mcc)
    
    def under_sampling(self):
        x=self.x
        y=self.y
        x_ture=x[y==True]
        x_false=x[y==False]
        y_ture=y[y==True]
        y_false=y[y==False]
        positive_n=len(y_ture)
        negative_n=len(y_false)
        random_index=np.random.randint(0,negative_n,positive_n)  
        x_false_u=x_false[random_index]
        y_false_u=y_false[random_index]
        return np.concatenate([x_ture, x_false_u]), np.concatenate([y_ture, y_false_u])
    
    def run(self):
        self.tprs=[]
        self.rocaucs=[]
        self.praucs=[]
        self.mccs=[]
        np.random.seed(seed=0)
        for i in range(10):
            x,y=self.under_sampling()
            tpr, rocauc, prauc, mcc = self.cv(x,y)
            self.tprs.append(tpr)
            self.rocaucs.append(rocauc)
            self.praucs.append(prauc)
            self.mccs.append(mcc)
            print("{}/10 Done".format(i+1))

In [ ]:
x=np.array(list_client+list_others)
y=np.array([True]*len(list_client) + [False]*len(list_others))
model=make_pipeline(StandardScaler(), SVC(class_weight="balanced", probability=True))

In [ ]:
svm_display=Display_auc(x,y,model)
svm_display.run()

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
fpr = np.linspace(0, 1, 1000)
for i, tpr in enumerate(svm_display.tprs):
    ax.plot(
    fpr,
    tpr,
    color="gray",
    lw=1,
    alpha=0.3
    )
mean_tpr = np.mean(svm_display.tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc=mean(svm_display.rocaucs)
std_auc=stdev(svm_display.rocaucs)
ax.plot(
    fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)
ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", alpha=0.8)
ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05]
)
plt.title("Seq2Phase", fontsize=20)
ax.legend(loc="lower right")
plt.show()

In [ ]:
print("{:.3f}±{:.3f}".format(mean(svm_display.rocaucs), stdev(svm_display.rocaucs)))
print("{:.3f}±{:.3f}".format(mean(svm_display.praucs), stdev(svm_display.praucs)))
print("{:.3f}±{:.3f}".format(mean(svm_display.mccs), stdev(svm_display.mccs)))

In [ ]:
client_set=set()
others_set=set()
for rec in SeqIO.parse("../fig1/result/drllps_client_clstr_Homo_sapiens.fasta", "fasta"):
    client_set.add(rec.id)
for rec in SeqIO.parse("../fig1/result/drllps_nonllps_clstr_Homo_sapiens.fasta", "fasta"):
    others_set.add(rec.id)
    
mat=np.load("human_feature.npy", allow_pickle=True)
mat=mat.item()

list_client=[]
list_others=[]
for k in mat.keys():
    if k in others_set:
        list_others.append(mat[k])
    elif k in client_set:
        list_client.append(mat[k])

In [ ]:
x=np.array(list_client+list_others)
y=np.array([True]*len(list_client) + [False]*len(list_others))
model=make_pipeline(StandardScaler(), SVC(class_weight="balanced", probability=True))

In [ ]:
svm_display_fea=Display_auc(x,y,model)
svm_display_fea.run()

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
fpr = np.linspace(0, 1, 1000)
for i, tpr in enumerate(svm_display_fea.tprs):
    ax.plot(
    fpr,
    tpr,
    color="gray",
    lw=1,
    alpha=0.3
    )
mean_tpr = np.mean(svm_display_fea.tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc=mean(svm_display_fea.rocaucs)
std_auc=stdev(svm_display_fea.rocaucs)
ax.plot(
    fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)
ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", alpha=0.8)
ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05]
)
plt.title("425 features", fontsize=20)
ax.legend(loc="lower right")
plt.show()

In [ ]:
print("{:.3f}±{:.3f}".format(mean(svm_display_fea.rocaucs), stdev(svm_display_fea.rocaucs)))
print("{:.3f}±{:.3f}".format(mean(svm_display_fea.praucs), stdev(svm_display_fea.praucs)))
print("{:.3f}±{:.3f}".format(mean(svm_display_fea.mccs), stdev(svm_display_fea.mccs)))

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)

fpr = np.linspace(0, 1, 1000)
nda=np.array(svm_display_fea.tprs)
for i, tpr in enumerate(nda):
    ax.plot(
    fpr,
    tpr,
    color="gray",
    lw=1,
    alpha=0.3
    )
mean_tpr = np.mean(nda, axis=0)
mean_tpr[-1] = 1.0
ax.plot(
    fpr,
    mean_tpr,
    color="cornflowerblue",
    label=r"425 features (AUC = 0.783 $\pm$ 0.005)",
    lw=2,
    alpha=1,
)

fpr = np.linspace(0, 1, 1000)
nda=np.array(svm_display.tprs)
for i, tpr in enumerate(nda):
    ax.plot(
    fpr,
    tpr,
    color="gray",
    lw=1.5,
    alpha=0.5
    )
mean_tpr = np.mean(nda, axis=0)
mean_tpr[-1] = 1.0
ax.plot(
    fpr,
    mean_tpr,
    color="b",
    label=r"Seq2Phase (AUC = 0.855 $\pm$ 0.003)",
    lw=2,
    alpha=0.8,
)

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", alpha=0.3)
ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05]
)


ax.set_xlabel("False positive rate", fontsize=13)
ax.set_ylabel("True positive rate", fontsize=13)
ax.set_title("Client vs. Non-LLPS", fontsize=15)
ax.legend(loc="lower right", fontsize=10)
plt.savefig("result/fig2a.pdf")
plt.show()